# MACD Strategy

**Stocks are traded using the macd strategy and optimized for the maximum Returns/Sharpe Ratio.**

1. Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import talib as ta
import yfinance as yf

import warnings
warnings.filterwarnings("ignore")

2. Retrieving stock data

In [2]:
# Stock data and parameters
ticker   = "TSLA"            
start    = "2022-03-01"      
end      = "2023-03-01"
interval = "1h"

df = yf.download(ticker, start=start, end=end, interval=interval, progress=False)
df.head()

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2022-03-01 09:30:00,289.893341,296.000000,285.565063,294.383331,294.383331,8254562
2022-03-01 10:30:00,294.546661,296.626678,288.766663,289.170013,289.170013,4280978
2022-03-01 11:30:00,289.088348,289.149994,285.880005,288.699982,288.699982,3190102
2022-03-01 12:30:00,288.643341,291.316681,287.000000,287.166656,287.166656,2212578
2022-03-01 13:30:00,287.208344,289.329987,286.253448,288.675018,288.675018,1707995


3. Cleaning the data

In [3]:
# Removing Extra Columns and Rows
df = df[["Open", "High", "Low", "Close", "Volume"]]
df.dropna(inplace=True)

df.head()

,Open,High,Low,Close,Volume
Datetime,,,,,
2022-03-01 09:30:00,289.893341,296.000000,285.565063,294.383331,8254562
2022-03-01 10:30:00,294.546661,296.626678,288.766663,289.170013,4280978
2022-03-01 11:30:00,289.088348,289.149994,285.880005,288.699982,3190102
2022-03-01 12:30:00,288.643341,291.316681,287.000000,287.166656,2212578
2022-03-01 13:30:00,287.208344,289.329987,286.253448,288.675018,1707995


4. Creating a Backtesting Strategy

In [4]:
# Importing Backtesting Libraries
from backtesting import Backtest
from backtesting.lib import crossover, TrailingStrategy

#   MACD Strategy :
#   ---------------
# - MACD and MACD_Signal indicators are calculated using fastperiod and slowperiod as interval periods.
# - When MACD passes up the Signal indicator, it triggers a buy! or a long position.
# - When MACD dips below the Signal indicator, it triggers a sell! or a sell position.
# - Stop loss and Take profit factors could be set as well, but Trailing stop loss is used to capture excess profit from trades.
# - Strategy also captures channel breakouts outside of couple Average True Range to place long and short positions respectively.


class MacdCross(TrailingStrategy):
    fastperiod = 12
    slowperiod = 26
    signalperiod = 9

    def init(self):
        super().init()
        self.set_trailing_sl(3)

        high   = self.data.High
        low    = self.data.Low                
        close  = self.data.Close
        self.macd, self.signal = self.I(ta.MACD, close, self.fastperiod, self.slowperiod, self.signalperiod)[:2]
        self.atr = self.I(ta.ATR, high, low, close, 14)
        self.upperChannel = np.roll(close, 1) + 3*self.atr
        self.lowerChannel = np.roll(close, 1) - 3*self.atr

    def next(self):
        super().next()

        if crossover(self.macd, self.signal) or crossover(self.data.Close, self.upperChannel):
            self.buy(size = 0.6)

        elif crossover(self.signal, self.macd) or crossover(self.data.Close, self.lowerChannel):
            self.sell(size = 0.6)

5. Executing the Strategy

In [5]:
bt = Backtest(df, MacdCross,
            cash=100000, trade_on_close=False, exclusive_orders=True)

# stats = bt.optimize(fastperiod = range(12, 40, 2),
#                     slowperiod = range(20, 60, 4),
#                     signalperiod = range(5, 15, 2),
#                     constraint = lambda p: p.slowperiod > p.fastperiod + 10 and p.fastperiod > p.signalperiod + 5,
#                     maximize = "Equity Final [$]")
stats = bt.run()

6. Comparing strategy returns to performance range of stocks

In [6]:
print("OC-Range: ", (df["Close"].iloc[-1]-df["Close"].iloc[0])/df["Close"].iloc[0])
print("HL-Range: ", (np.max(df["Close"])-np.min(df["Close"]))/np.min(df["Close"]))

OC-Range:  -0.30118325547489294
HL-Range:  2.61565092302896


7. Optimized Strategy Statistics

In [7]:
stats

Start                     2022-03-01 09:30:00
End                       2023-02-28 15:30:00
Duration                    364 days 06:00:00
Exposure Time [%]                   92.132269
Equity Final [$]                 84848.756298
Equity Peak [$]                 126943.905003
Return [%]                         -15.151244
Buy & Hold Return [%]              -30.118326
Return (Ann.) [%]                  -15.206766
Volatility (Ann.) [%]               31.987095
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -34.655837
Avg. Drawdown [%]                   -5.006851
Max. Drawdown Duration      160 days 01:00:00
Avg. Drawdown Duration       19 days 13:00:00
# Trades                                  188
Win Rate [%]                        40.425532
Best Trade [%]                      23.670181
Worst Trade [%]                    -11.466102
Avg. Trade [%]                    

8. Hyperparameters used by the Optimized Strategy that can be used to trade further

In [8]:
print("Fastperiod  : ", stats._strategy.fastperiod)
print("Slowperiod  : ", stats._strategy.slowperiod)
print("Signalperiod: ", stats._strategy.signalperiod)

Fastperiod  :  12
Slowperiod  :  26
Signalperiod:  9


In [9]:
plt = bt.plot(results=stats)

In [10]:
stats._trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,214,38,52,279.673340,274.253326,-1159.882874,-0.019380,2022-03-08 12:30:00,2022-03-10 12:30:00,2 days 00:00:00
1,-216,52,72,274.253326,266.266663,1725.119385,0.029121,2022-03-10 12:30:00,2022-03-15 11:30:00,4 days 23:00:00
2,226,72,116,266.266663,329.292463,14243.830873,0.236702,2022-03-15 11:30:00,2022-03-23 13:30:00,8 days 02:00:00
3,-204,119,133,336.576660,355.033325,-3765.159668,-0.054836,2022-03-24 09:30:00,2022-03-28 09:30:00,4 days 00:00:00
4,183,135,146,362.770020,366.818329,740.840607,0.011159,2022-03-28 11:30:00,2022-03-29 15:30:00,1 days 04:00:00
...,...,...,...,...,...,...,...,...,...,...
183,-269,1736,1738,195.949997,195.740005,56.487701,0.001072,2023-02-24 12:30:00,2023-02-24 14:30:00,0 days 02:00:00
184,-270,1738,1739,195.740005,196.199997,-124.197693,-0.002350,2023-02-24 14:30:00,2023-02-24 15:30:00,0 days 01:00:00
185,-269,1739,1740,196.199997,202.029999,-1568.270493,-0.029715,2023-02-24 15:30:00,2023-02-27 09:30:00,2 days 18:00:00
186,-256,1740,1741,202.029999,207.009995,-1274.878906,-0.024650,2023-02-27 09:30:00,2023-02-27 10:30:00,0 days 01:00:00
